# Model Training

In [1]:
# IMPORT LIBRARIES
import argparse
import glob
from itertools import chain, combinations, product
import math
import importlib
import os
import random
import pickle
import copy

# IMPORT STANDARD PACKAGES
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
from scipy.signal import welch, firwin
from scipy import stats
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import balanced_accuracy_score

# IMPORT TORCH
import torch
import torch.nn as nn
from torchaudio import transforms
from torch.utils.data import DataLoader

# IMPORT SELFEEG 
import selfeeg
import selfeeg.models as zoo
import selfeeg.dataloading as dl

# IMPORT REPOSITORY FUNCTIONS
import AllFnc
from AllFnc import split
from AllFnc.models import ShallowNet2
from AllFnc.training import (
    loadEEG,
    lossBinary,
    lossMulti,
    train_model,
    get_performances,
    GetLearningRate)

# IMPORT EEGVISLIB
from AllFnc import eegvislib

import warnings
warnings.filterwarnings("ignore", message = "Using padding='same'", category = UserWarning)

In [2]:
# SET DEVICE: 0 = ZANOLA
if torch.cuda.is_available():
    device = torch.device(0)
print(device)

# SET PATH
dataPath = '/data/delpup/datasets/eegpickle/'
osPath   = os.path.abspath(os.getcwd())
sep      = os.path.sep
imgPath  = osPath + sep + 'imgs' + sep

cuda:0


## Set training spec

In [3]:
pipelineToEval = 'filt' #'icasr'
taskToEval     = 'alzheimer'
modelToEval    = 'shallownet_custom' #shallownet, shallownet_custom 
outFold        = 6
inFold         = 5
downsample     = True
z_score        = True
rem_interp     = True
batchsize      = 64
overlap        = 0.0
workers        = 0
window         = 4
verbose        = True
lr             = 5*10**(-5)
epochs         = 1000
lossVal        = None
arch_acronym   = 'shn7db'
seed           = 83136297

# ShallowNet
# shallownet_custom_dict = {
#     "F1": 40,
#     "K1": 25,
#     "F2": 40,
#     "Pool": 75,
#     "p": 0.2,
#     "log_activation_base": "e",
#     "norm_type": "batchnorm",
#     "random_temporal_filter": True,
#     "Fs": 125 if downsample else 250 ,
#     "freeze_temporal": 0,
#     "dense_hidden": None,
#     "spatial_depthwise": False,
#     "spatial_only_positive": False,
#     "global_pooling": False,
#     "return_logits": True,
#     "seed": 83136297
# }

# Med-ShallowNet
shallownet_custom_dict = {
    "F1": 7,
    "K1": 125,
    "F2": 7,
    "Pool": 75,
    "p": 0.2,
    "log_activation_base": "dB",
    "norm_type": "batchnorm",
    "random_temporal_filter": False,
    "Fs": 125 if downsample else 250 ,
    "freeze_temporal": 999999999,
    "dense_hidden": None,
    "spatial_depthwise": True,
    "spatial_only_positive": False,
    "global_pooling": True,
    "bias": [False, False, False],
    "return_logits": True,
    "seed": seed
}

In [4]:
# fold to eval is the correct index to get the desired train/val/test part
outerFold  = outFold - 1
innerFold  = inFold  - 1
foldToEval = outerFold*5 + innerFold

## Create partition list

In [5]:
if 'alzheimer' in taskToEval.casefold():
    # ALZ = subjects 1 to 36; CTL = subjects 37 to 65; FTD = subjects 66 to 88
    a_id = [i for i in range(1,37)]
    c_id = [i for i in range(37,66)]
    f_id = [i for i in range(66,89)]
    
    part_a = split.create_nested_kfold_subject_split(a_id, 10, 5)
    part_c = split.create_nested_kfold_subject_split(c_id, 10, 5)
    part_f = split.create_nested_kfold_subject_split(f_id, 10, 5)

    if taskToEval.casefold() == 'alzheimer':
        partition_list_1 = split.merge_partition_lists(part_a, part_c, 10, 5)
        partition_list = split.merge_partition_lists(partition_list_1, part_f, 10, 5)
    elif taskToEval.casefold() == 'alzheimerca':
        partition_list = split.merge_partition_lists(part_a, part_c, 10, 5)
    elif taskToEval.casefold() == 'alzheimercf':
        partition_list = split.merge_partition_lists(part_c, part_f, 10, 5)
    elif taskToEval.casefold() == 'alzheimeraf':
        partition_list = split.merge_partition_lists(part_a, part_f, 10, 5)

elif taskToEval.casefold() == 'cognitive':
    # CTL = subjects 101 to 149; PD/PDD/PDMCI = mixing number in [1; 100]
    c_id = [i for i in range(101,150)]
    pd_id = [3, 6, 7, 16, 17, 18, 21, 24, 26, 27, 30, 32, 35, 37, 39, 40, 45,
             46, 50, 51, 53, 57, 58, 60, 61, 62, 63, 65, 67, 68, 69, 71, 74,
             76, 80, 81, 82, 83, 84, 85, 86, 87, 90, 92, 93, 94, 100]
    pdd_id = [1, 2, 5, 8, 9, 12, 13, 15, 22, 25, 33, 38, 44, 48, 75, 78, 88, 95, 96]
    pdmci_id = [4, 10, 11, 14, 19, 20, 23, 28, 29, 31, 34, 36, 41, 42, 43, 47,
                49, 52, 54, 55, 56, 59, 64, 66, 70, 72, 73, 77, 79, 89, 91, 97,
                98, 99]
    part_c = split.create_nested_kfold_subject_split(c_id, 10, 5)
    part_p = split.create_nested_kfold_subject_split(pd_id, 10, 5)
    part_d = split.create_nested_kfold_subject_split(pdd_id, 10, 5)
    part_m = split.create_nested_kfold_subject_split(pdmci_id, 10, 5)
    
    # first --> mix two groups; then --> mix the mix
    # splits have a more similar number of subject per set in this way
    partition_1 = split.merge_partition_lists(part_c, part_m, 10, 5)
    partition_2 = split.merge_partition_lists(part_p, part_d, 10, 5)
    partition_list = split.merge_partition_lists(partition_1, partition_2, 10, 5)

## Get other dataset info

In [6]:
# Define the Path to EEG data as a concatenation of:
# 1) the root path
# 2) the preprocessing pipeline
if dataPath[-1] != os.sep:
    dataPath += os.sep
if pipelineToEval[-1] != os.sep:
    eegpath = dataPath + pipelineToEval + os.sep
else:
    eegpath = dataPath + pipelineToEval

# Define the number of Channels to use. 
# Basically 61 due to BIDSAlign channel system alignment.
# Note that BIDSAlign DOES NOT delete any original channel by default.
if rem_interp:
    if 'alzheimer' in taskToEval.casefold():
        Chan = 19
    elif taskToEval.casefold() == 'cognitive':
        Chan = 59
else:
    Chan = 61

# Define the sampling rate. 125 or 250 depending on the downsample option
srate = 125 if downsample else 250

# Define the number of classes to predict.
# All tasks are binary except the Alzheimer's one, 
# which is a multi-class classification (Alzheimer vs FrontoTemporal vs Control)
if taskToEval.casefold() == 'alzheimer':
    nb_classes = 3
elif taskToEval.casefold() == 'cognitive':
    nb_classes = 4
else:
    nb_classes = 2

# For selfEEG's models instantiation
Samples = int(srate*window)

# Set the Dataset ID for glob.glob operation in SelfEEG's GetEEGPartitionNumber().
if 'alzheimer' in taskToEval.casefold():
    datasetID = '10'
elif taskToEval.casefold() == 'cognitive':
    datasetID = '19'

# Set the class label in case of plot of functions
if taskToEval.casefold() == 'alzheimer':
    classlabels = ['CTL', 'FTD', 'AD']
elif taskToEval.casefold() == 'alzheimerca':
    classlabels = ['CTL', 'AD']
elif taskToEval.casefold() == 'alzheimercf':
    classlabels = ['CTL', 'FTD']
elif taskToEval.casefold() == 'alzheimeraf':
    classlabels = ['FTD', 'AD']
elif taskToEval.casefold() == 'cognitive':
    classlabels = ['CTL', 'PD', 'PDD', 'PDMCI']

## Define torch dataloaders

In [7]:
loadEEG_args = {'return_label': False, 
                'downsample': downsample, 
                'use_only_original': rem_interp,
                'apply_zscore': z_score}

glob_input = [datasetID + '_*.pickle']

# calculate dataset length.
# Basically it automatically retrieves all the partitions 
# that can be extracted from each EEG signal
EEGlen = dl.get_eeg_partition_number(
    eegpath,
    srate,
    window,
    overlap, 
    file_format = glob_input,
    load_function = loadEEG,
    optional_load_fun_args = loadEEG_args,
    includePartial = False if overlap == 0 else True,
    verbose = verbose
)

# Now we also need to load the labels
loadEEG_args['return_label'] = True

# Set functions to retrieve dataset, subject, and session from each filename.
# They will be used by GetEEGSplitTable to perform a subject based split
dataset_id_ex  = lambda x: int(x.split(os.sep)[-1].split('_')[0])
subject_id_ex  = lambda x: int(x.split(os.sep)[-1].split('_')[1]) 
session_id_ex  = lambda x: int(x.split(os.sep)[-1].split('_')[2]) 

# Now call the GetEEGSplitTable. 
if taskToEval.casefold() == 'alzheimerca':
    exclude_id = f_id
elif taskToEval.casefold() == 'alzheimercf':
    exclude_id = a_id
elif taskToEval.casefold() == 'alzheimeraf':
    exclude_id = c_id
else:
    exclude_id = None
        
    train_id   = partition_list[foldToEval][0]
    val_id     = partition_list[foldToEval][1]
    test_id    = partition_list[foldToEval][2]
    EEGsplit = dl.get_eeg_split_table(
        partition_table      = EEGlen,
        val_data_id          = val_id,
        test_data_id         = test_id,
        exclude_data_id      = exclude_id,
        split_tolerance      = 0.001,
        dataset_id_extractor = subject_id_ex,
        subject_id_extractor = session_id_ex,
        perseverance         = 10000)

if verbose:
    print(' ')
    print('Subjects used for test')
    print(test_id)

extracting EEG samples: 100%|███████████████| 88/88 [00:04<00:00, 21.64 files/s]

Concluded extraction of repository length with the following specific: 

window          ==>  4.00 s
overlap         ==>  0.00 %
sampling rate   ==> 125.00 Hz
-----------------------------
dataset length  ==>    17604
 
Subjects used for test


NameError: name 'test_id' is not defined

In [8]:
# Define Datasets and preload all data
trainset = dl.EEGDataset(
    EEGlen, EEGsplit, [srate, window, overlap], 'train', 
    supervised             = True, 
    label_on_load          = True,
    load_function          = loadEEG,
    optional_load_fun_args = loadEEG_args
)
trainset.preload_dataset()

valset = dl.EEGDataset(
    EEGlen, EEGsplit, [srate, window, overlap], 'validation',
    supervised             = True, 
    label_on_load          = True,
    load_function          = loadEEG,
    optional_load_fun_args = loadEEG_args
)
valset.preload_dataset()

testset = dl.EEGDataset(
    EEGlen, EEGsplit, [srate, window, overlap], 'test',
    supervised             = True,
    label_on_load          = True,
    load_function          = loadEEG,
    optional_load_fun_args = loadEEG_args
)
testset.preload_dataset()

# Convert to long if task is multiclass classification.
# This avoids Value Errors during cross entropy loss calculation
if ('alzheimer' in taskToEval.casefold()) or ('cognitive' in taskToEval.casefold()):
    if taskToEval.casefold() == 'alzheimerca':
        trainset.y_preload[trainset.y_preload==2] = 1
        valset.y_preload[valset.y_preload==2] = 1 
        testset.y_preload[testset.y_preload==2] = 1
        
    elif taskToEval.casefold() == 'alzheimeraf':
        trainset.y_preload -= 1
        valset.y_preload   -= 1 
        testset.y_preload  -= 1
        
    elif taskToEval.casefold() == 'alzheimeraf':
        pass
        
    else:
        trainset.y_preload = trainset.y_preload.to(dtype = torch.long)
        valset.y_preload   = valset.y_preload.to(dtype = torch.long)
        testset.y_preload  = testset.y_preload.to(dtype = torch.long)
    
# Finally, Define Dataloaders
# (no need to use more workers in validation and test dataloaders)
trainloader = DataLoader(dataset = trainset, batch_size = batchsize,
                         shuffle = True, num_workers = workers)
valloader = DataLoader(dataset = valset, batch_size = batchsize,
                       shuffle = False, num_workers = 0)
testloader = DataLoader(dataset = testset, batch_size = batchsize,
                        shuffle = False, num_workers = 0)

if verbose:
    # plot split statistics
    labels = np.zeros(len(EEGlen))
    for i in range(len(EEGlen)):
        path = EEGlen.iloc[i,0]
        with open(path, 'rb') as eegfile:
            EEG = pickle.load(eegfile)
        labels[i] = EEG['label']
    dl.check_split(EEGlen, EEGsplit, labels)


train ratio:      0.75
validation ratio: 0.16
test ratio:       0.10

train labels ratio: 0.0 = 0.341 ,  1.0 = 0.220 ,  2.0 = 0.439 , 
val   labels ratio: 0.0 = 0.364 ,  1.0 = 0.265 ,  2.0 = 0.371 , 
test  labels ratio: 0.0 = 0.360 ,  1.0 = 0.324 ,  2.0 = 0.316 , 



## Define loss, model, and other training parameter

In [9]:
validation_loss_args = []
if taskToEval.casefold()=="alzheimer" or ('cognitive' in taskToEval.casefold()):
    lossFnc = lossMulti
else:
    lossFnc = lossBinary

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# define model
Mdl = ShallowNet2(nb_classes, Chan, Samples, **shallownet_custom_dict)
MdlBase = copy.deepcopy(Mdl)

Mdl.to(device = device)
Mdl.train()
if verbose:
    print(' ')
    ParamTab = selfeeg.utils.count_parameters(Mdl, False, True, True)
    print(' ')

if lr == 0:
    lr = GetLearningRate(modelToEval, taskToEval)
    if verbose:
        print(' ')
        print('used learning rate', lr)
        
optimizer = torch.optim.Adam(Mdl.parameters(), lr = lr)
gamma = 0.995
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = gamma)

#factor = 0.5
#patience_scheduler = 10
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=factor, patience=patience_scheduler)

# Define selfEEG's EarlyStopper with large patience to act as a model checkpoint
earlystop = selfeeg.ssl.EarlyStopping(
    patience = 15, 
    min_delta = 1e-04, 
    record_best_weights = True
)

 
                 Modules  Parameters
0   encoder.conv1.weight        1000
1     encoder.conv1.bias          40
2   encoder.conv2.weight       30400
3     encoder.conv2.bias          40
4  encoder.batch1.weight          40
5    encoder.batch1.bias          40
6           Dense.weight        3240
7             Dense.bias           3
  TOTAL TRAINABLE PARAMS       34803
 


## Train the model

In [10]:
loss_summary=train_model(model                 = Mdl,
                        train_dataloader      = trainloader,
                        epochs                = epochs,
                        optimizer             = optimizer,
                        loss_func             = lossFnc, 
                        lr_scheduler          = scheduler,
                        EarlyStopper          = earlystop,
                        validation_dataloader = valloader,
                        validation_loss_func  = lossVal,
                        validation_loss_args  = validation_loss_args,
                        verbose               = verbose,
                        device                = device,
                        return_loss_info      = True
                        )

epoch [1/1000]
Epoch 0: Balanced Accuracy = 50.89% | 205/250 [75.18 Batch/s, train_loss=0.93351, val_loss=0.00000]
   val 44/44: 100%|███████████████| 250/250 [66.59 Batch/s, train_loss=0.93351, val_loss=0.88031]  
epoch [2/1000]
Epoch 1: Balanced Accuracy = 64.69% | 205/250 [72.87 Batch/s, train_loss=0.69728, val_loss=0.00000]
   val 44/44: 100%|███████████████| 250/250 [82.85 Batch/s, train_loss=0.69728, val_loss=0.95355]  
epoch [3/1000]
Epoch 2: Balanced Accuracy = 74.71% | 205/250 [75.11 Batch/s, train_loss=0.53478, val_loss=0.00000]
   val 44/44: 100%|███████████████| 250/250 [86.91 Batch/s, train_loss=0.53478, val_loss=0.99563]  
epoch [4/1000]
Epoch 3: Balanced Accuracy = 82.63% | 205/250 [77.97 Batch/s, train_loss=0.41545, val_loss=0.00000]
   val 44/44: 100%|███████████████| 250/250 [89.30 Batch/s, train_loss=0.41545, val_loss=1.07336]  
epoch [5/1000]
Epoch 4: Balanced Accuracy = 87.61% | 205/250 [74.65 Batch/s, train_loss=0.33141, val_loss=0.00000]
   val 44/44: 100%|██████

## Evaluate the model

In [ ]:
earlystop.restore_best_weights(Mdl)
Mdl.to(device=device)
Mdl.eval()
scores = get_performances(loader2eval    = testloader, 
                          Model          = Mdl, 
                          device         = device,
                          nb_classes     = nb_classes,
                          return_scores  = True,
                          verbose        = verbose,
                          plot_confusion = True,
                          class_labels   = classlabels
                         )
print(f"Balanced Accuracy: {scores['accuracy_weighted']*100: .2f} %")

In [ ]:
training_loss_curve = []
validation_loss_curve = []
# Iterate through the dictionary, filtering out entries with None values
for key, (train_loss, val_loss) in loss_summary.items():
    if train_loss is not None and val_loss is not None:
        training_loss_curve.append(train_loss)
        validation_loss_curve.append(val_loss)
        
scores['training_loss_curve']   = training_loss_curve
scores['validation_loss_curve'] = validation_loss_curve

plt.plot(training_loss_curve,   label='Training Loss')
plt.plot(validation_loss_curve, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Cross-Entropy Loss')
plt.legend()
plt.show()

In [ ]:
Mdl_weights = {k: v.cpu() for k, v in Mdl.state_dict().items()}

# Convert to a regular dictionary if needed
Mdl_weights = dict(Mdl_weights)

## Initial Model

In [ ]:
MdlBase.to(device=device)
MdlBase.eval()
scoresBase = get_performances(loader2eval    = testloader, 
                              Model          = MdlBase, 
                              device         = device,
                              nb_classes     = nb_classes,
                              return_scores  = True,
                              verbose        = verbose,
                              plot_confusion = True,
                              class_labels   = classlabels
                             )

In [ ]:
MdlBase_weights = {k: v.cpu() for k, v in MdlBase.state_dict().items()}

# Convert to a regular dictionary if needed
MdlBase_weights = dict(MdlBase_weights)

## Comparison with Statistical Methods

In [ ]:
layer = 'encoder'
Mdl.to('cpu')

test_window_normalized       = testloader.dataset[:][0].to( device = 'cpu')
training_window_normalized   = trainloader.dataset[:][0].to(device = 'cpu')
validation_window_normalized = valloader.dataset[:][0].to(  device = 'cpu')
y_train = trainloader.dataset[:][1].to(device = 'cpu').numpy()
y_test  = testloader.dataset[:][1].to(device = 'cpu').numpy()

f_desired = [1, 4, 8, 12, 16, 20, 28, 45]

In [ ]:
f,Pxx = eegvislib.get_Pxx(test_window_normalized[0,:,:],srate=srate)
findex = eegvislib.get_freq_index(f, f_desired)
band_content = np.zeros((test_window_normalized.shape[0],shallownet_custom_dict['F1']))
for i in range(test_window_normalized.shape[0]):
    f,Pxx = eegvislib.get_Pxx(test_window_normalized[i,:,:],srate=srate)

    for j in range(shallownet_custom_dict['F1']):
        #bandc = np.log10(np.mean(np.trapz(Pxx[findex[j]:findex[j+1],:].T,dx=(f[1]-f[0]))/(f_desired[j+1]-f_desired[j])))
        bandc = 10*np.log10(np.mean(np.trapz(Pxx[findex[j]:findex[j+1],:].T,dx=(f[1]-f[0]))/1))
        band_content[i,j] = bandc

f,Pxx = eegvislib.get_Pxx(training_window_normalized[0,:,:],srate=srate)
band_content_training = np.zeros((training_window_normalized.shape[0],shallownet_custom_dict['F1']))
for i in range(training_window_normalized.shape[0]):
    f,Pxx = eegvislib.get_Pxx(training_window_normalized[i,:,:],srate=srate)

    for j in range(shallownet_custom_dict['F1']):
        #bandc = np.log10(np.mean(np.trapz(Pxx[findex[j]:findex[j+1],:].T,dx=(f[1]-f[0]))/(f_desired[j+1]-f_desired[j])))
        bandc = 10*np.log10(np.mean(np.trapz(Pxx[findex[j]:findex[j+1],:].T,dx=(f[1]-f[0]))/1))
        band_content_training[i,j] = bandc

### Multinomial Logistic Regression Model (MLRM)

In [ ]:
X_train = band_content_training
X_test  = band_content

print('Reference Class: 0 = CTL')
model = sm.MNLogit(y_train, X_train)
result = model.fit(method='ncg')

print(result.summary())

# Predict on the test data
predictions = result.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
accuracy = balanced_accuracy_score(y_test, predicted_classes)
print(f"Balanced Accuracy: {accuracy*100: .2f} %")

### shnMLRM: ShallowNet Encoder + MLRM

In [ ]:
convoluted_output_test       = eegvislib.out_activation(Mdl, layer, test_window_normalized)
convoluted_output_val        = eegvislib.out_activation(Mdl, layer, validation_window_normalized)
convoluted_output_train      = eegvislib.out_activation(Mdl, layer, training_window_normalized)

In [ ]:
X_train = convoluted_output_train.to(device = 'cpu').numpy()
X_test  = convoluted_output_test.to(device = 'cpu').numpy()

print('Reference Class: 0 = CTL')
model = sm.MNLogit(y_train, X_train)
result = model.fit(method='ncg')

print(result.summary())

# Predict on the test data
predictions = result.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
accuracy = balanced_accuracy_score(y_test, predicted_classes)
print(f"Balanced Accuracy: {accuracy*100: .2f} %")

## Overlap Calculation

In [ ]:
# Process test, validation, and training data only once and stay on GPU if possible
layer = 'Dense'
Mdl.to('cpu')
datasets           = [testloader, valloader, trainloader]
convoluted_outputs = [eegvislib.out_activation(Mdl, layer, ds.dataset[:][0].to(device='cpu')) for ds in datasets]
embeddingF         = torch.cat(convoluted_outputs, dim=0).cpu().numpy()

test_samples       = len(testloader.dataset[:][1])
val_samples        = len(valloader.dataset[:][1])
train_samples      = len(trainloader.dataset[:][1])

# Constants and settings
prob = 0.6827
grid_size = 50j  # Grid resolution for KDE calculation

overlap_traintest = eegvislib.get_overlap3D(embeddingF, [test_samples + val_samples,-1], [0,test_samples],
                                            prob=prob, grid_size=grid_size)

overlap_trainval  = eegvislib.get_overlap3D(embeddingF, [test_samples + val_samples,-1], [test_samples, test_samples + val_samples],
                                            prob=prob, grid_size=grid_size)

overlap_valtest   = eegvislib.get_overlap3D(embeddingF, [test_samples, test_samples + val_samples], [0, test_samples],
                                            prob=prob, grid_size=grid_size)

print('Overlap between Sets in the logit space')
print(f'Overlap train-test: {overlap_traintest:.2%}, Overlap train-val: {overlap_trainval:.2%}, Overlap val-test: {overlap_valtest:.2%}')

## Save the Model

In [ ]:
# if taskToEval.casefold() == 'alzheimer':
#     start_piece_mdl = 'AlzClassification/Models/'
#     start_piece_res = 'AlzClassification/Results/'
#     task_piece = 'alz'
# elif taskToEval.casefold() == 'alzheimerca':
#     start_piece_mdl = 'AlzCAClassification/Models/'
#     start_piece_res = 'AlzCAClassification/Results/'
#     task_piece = 'al1'
# elif taskToEval.casefold() == 'alzheimer':
#     start_piece_mdl = 'AlzCFClassification/Models/'
#     start_piece_res = 'AlzCFClassification/Results/'
#     task_piece = 'al2'
# elif taskToEval.casefold() == 'alzheimer':
#     start_piece_mdl = 'AlzAFClassification/Models/'
#     start_piece_res = 'AlzAFClassification/Results/'
#     task_piece = 'al3'

# mdl_piece = modelToEval.casefold()
    
# if pipelineToEval.casefold() == 'raw':
#     pipe_piece = 'raw'
# elif pipelineToEval.casefold() == 'filt':
#     pipe_piece = 'flt'
# elif pipelineToEval.casefold() == 'ica':
#     pipe_piece = 'ica'
# elif pipelineToEval.casefold() == 'icasr':
#     pipe_piece = 'isr'

# if downsample:
#     freq_piece = '125'
# else:
#     freq_piece = '250'

# out_piece = str(outerFold+1).zfill(3)
# in_piece = str(innerFold+1).zfill(3)
# lr_piece = str(int(lr*1e6)).zfill(6)
# chan_piece = str(Chan).zfill(3)
# win_piece = str(round(window)).zfill(3)

# file_name = '_'.join(
#     [task_piece, pipe_piece, freq_piece, mdl_piece, 
#      out_piece, in_piece, lr_piece, chan_piece, win_piece,
#      subj_piece, loss_piece, head_piece
#     ]
# )
# model_path = start_piece_mdl + file_name + '.pt'
# results_path = start_piece_res + file_name + '.pickle'

# if verbose:
#     print('saving model and results in the following paths')
#     print(model_path)
#     print(results_path)

# # Save the model
# Mdl.eval()
# Mdl.to(device='cpu')
# torch.save(Mdl.state_dict(), model_path)

# # Save the scores
# with open(results_path, 'wb') as handle:
#     pickle.dump(scores, handle, protocol = pickle.HIGHEST_PROTOCOL)

# if verbose:
#     print('run complete')